# 10. data preprocessing to save time in application-level evaluations

In [1]:
import uavgeo as ug
import geopandas as gpd
import numpy as np
import xarray as xr
import rioxarray as rxr
import matplotlib.pyplot as plt
import os
import pandas as pd

c:\users\judoj\documents\programming\uavgeo\uavgeo\compute\products.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## 2. CAN2023 preprocessing

In [3]:
# load data can2023


dsm = rxr.open_rasterio("../data/dsm.tif",mode = "r", chunks=True).astype(np.float32)
dsm = dsm.rio.write_nodata(dsm.rio.nodata, encoded=True)

rgb = rxr.open_rasterio("../data/rgb_230609.tif",mode = "r", chunks=True).astype(np.uint8)
rgb = rgb.rio.write_nodata(rgb.rio.nodata, encoded=True)
#and the mask
mask = gpd.read_file("../data/vineyard_shape.geojson")

In [4]:
rgb = rgb.rio.reproject("EPSG:32631")
dsm = dsm.rio.reproject("EPSG:32631")

In [5]:
#cset nodata value
dsm = dsm.rio.write_nodata(dsm.rio.nodata, encoded=True)
rgb = rgb.rio.write_nodata(rgb.rio.nodata, encoded=True)

C:\Users\judoj\mambaforge\envs\ndvi\Lib\site-packages\rioxarray\raster_writer.py:130: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(


In [ ]:
# align to same pixel-grid
#pixel-level alignment 
dsm = dsm.rio.reproject_match(rgb)

In [ ]:
# Calculate products DEM and CHM from DSM:

pixel_size = dsm.rio.resolution()[0]
sampling_meters = 1

#function will take a bit longer than the progress bar shows
dem = ug.compute.calc_dem_from_dsm(dsm, pixel_size, sampling_meters)
chm = dsm-dem

In [ ]:
# calculate it from the orthomosaic, or load it in here. 
ndvi = ug.compute.calc_ndvi(rgb, red_id = 1, nir_id= 2)

# or load it:
ndvi = rxr.open_rasterio("../data/ndvi.tif")

In [ ]:
mask = mask.to_crs("EPSG:32631")
mask = mask.geometry

In [ ]:
#mask it out
ndvi_ = ndvit.rio.clip(mask)

#mask 
chm= chm.rio.clip(mask)
dem = dem.rio.clip(mask)
dsm = dsm.rio.clip(mask)

rgb = rgb.rio.clip(mask)

In [ ]:
#cset nodata value again
ndvi = ndvi.rio.write_nodata(ndvi.rio.nodata, encoded=True)
dem = dem.rio.write_nodata(dem.rio.nodata, encoded=True)
chm = chm.rio.write_nodata(chm.rio.nodata, encoded=True)
dsm = dsm.rio.write_nodata(dsm.rio.nodata, encoded=True)
rgb= rgb.rio.write_nodata(rgb.rio.nodata, encoded = True)

In [ ]:
#save it
rgb.rio.to_raster("../data/canyelles/orthos/preprocessed/ndvi_test.tif")
ndvi.rio.to_raster("../data/preprocessed/ndvi.tif")
chm.rio.to_raster("../data/preprocessed/chm.tif")
dem.rio.to_raster("../data/preprocessed/dem.tif")
dsm.rio.to_raster("../data/preprocessed/dsm.tif")
